In [28]:
# Imports using Sklearn make shortcut functions
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery
from datetime import datetime, timedelta
import os


In [29]:
# Load csvs from raw_data folder
trends1 = pd.read_csv('../processed_data/LAB_CON_LIB_GRE_BRX_Trends_2004_now.csv', skiprows=1)
trends2 = pd.read_csv('../processed_data/LAB_PLC_SNP_UKI_NAT_Trends_2004_now.csv', skiprows=1)

In [30]:
trends2.columns

Index(['Month', 'Labour Party: (United Kingdom)',
       'Plaid Cymru: (United Kingdom)',
       'Scottish National Party: (United Kingdom)',
       'UK Independence Party: (United Kingdom)',
       'British National Party: (United Kingdom)'],
      dtype='object')

In [31]:
# Drop duplicate Labour column
trends2.drop(columns='Labour Party: (United Kingdom)',inplace=True)

In [32]:
# Join two trends datasets on 'Month'
trends_merged = pd.merge(trends1,trends2,how='left',on='Month')

In [33]:
# Convert 'Month' field to pd.datetime64[ns] format for joining with polling data
trends_merged['Month'] = pd.to_datetime(trends_merged['Month'])

In [34]:
# Replace <1 values with 0.5 value
trends_merged.replace('<1', 0.5,inplace=True)

In [35]:
# Converting objects (pd.Series) to int dtypes
trends_merged = trends_merged.astype({'Green Party: (United Kingdom)':'int','Reform UK: (United Kingdom)':'int',\
    'Plaid Cymru: (United Kingdom)':'int','Scottish National Party: (United Kingdom)':'int',\
        'UK Independence Party: (United Kingdom)':'int','British National Party: (United Kingdom)':'int'})

In [36]:
# Rename columns to align them more easily with _FC and _ACT columns
trends_merged.rename(columns={'Labour Party: (United Kingdom)': 'LAB_trends',
        'Conservative Party: (United Kingdom)': 'CON_trends',
        'Liberal Democrats: (United Kingdom)': 'LIB_trends',
        'Green Party: (United Kingdom)': 'GRE_trends',
        'Reform UK: (United Kingdom)': 'BRX_trends',
        'Plaid Cymru: (United Kingdom)': 'PLC_trends',
        'Scottish National Party: (United Kingdom)': 'SNP_trends',
        'UK Independence Party: (United Kingdom)': 'UKI_trends',
        'British National Party: (United Kingdom)': 'NAT_trends'}
        ,inplace=True)

In [37]:
ons = pd.read_csv('../processed_data/ons_economic_data_master.csv')

In [38]:
# Print current time
now = datetime.now()
current_time = now.strftime("%Y-%m-%d %H:%M:%S")

In [39]:
# Imports data
data = pd.read_csv('../processed_data/1988_to_2024_combined_clean_polling_and_results.csv')

In [40]:
# Convert date columns to datetime
data['enddate'] = pd.to_datetime(data['enddate'])
data['next_elec_date'] = pd.to_datetime(data['next_elec_date'])
data['startdate'] = pd.to_datetime(data['startdate'])

In [41]:
# Create poll enddate field with YYYY-MM format, so we can join with Trends data
data['enddate_year_month'] = pd.to_datetime(data['enddate']).dt.to_period('M')

In [42]:
# Convert Period datatype to str and then to datetime64
data.enddate_year_month = pd.to_datetime(data.enddate_year_month.astype('str'))

In [43]:
# Merge data with trends data, to get a df with polls, election results and trends
data = pd.merge(data,trends_merged,how='left',left_on='enddate_year_month',right_on='Month')

In [44]:
ons['Month'] = pd.to_datetime(ons['Month'])

In [45]:
# Merge data with trends data, to get a df with polls, election results and trends
data = pd.merge(data,ons,how='left',left_on='enddate_year_month',right_on='Month')

In [46]:
# Scale trends columns by diving by 100
for column in ['LAB_trends', 'CON_trends', 'LIB_trends',
       'GRE_trends', 'BRX_trends', 'PLC_trends', 'SNP_trends', 'UKI_trends',
       'NAT_trends']:
    data[column] = data[column] / 100

In [47]:
# Handle numerical transformer
num_columns_selector = ['samplesize', 'months_to_elec_weight', 'GDP','Inflation','Unemployment']
num_transformer = MinMaxScaler()

In [48]:
# Handle categorical transformer
cat_columns_selector = ['rating']
cat_transformer = make_pipeline(OrdinalEncoder(categories = [['F','F+','E-','E','E+','D-','D','D+','C-','C','C+','B-','B','B+','A-']]),MinMaxScaler())

In [49]:
# One-hot encoder for party_in_power
ohe = OneHotEncoder()

In [50]:
# Build the preprocessing pipeline
preproc_pipeline = make_column_transformer(
    (num_transformer, num_columns_selector),
    (cat_transformer, cat_columns_selector),
    (ohe,['party_in_power']),
    remainder='passthrough',
    verbose_feature_names_out=False
)

In [51]:
data['next_elec_date'].value_counts()

next_elec_date
2015-05-07    1930
2024-07-04    1397
2010-05-06     584
1992-04-09     453
2019-12-12     440
1997-05-01     338
2017-06-08     254
2001-06-07     213
2005-05-05      98
Name: count, dtype: int64

In [52]:
# Impute Inflation NaN values
data.Inflation.fillna(method='ffill',inplace=True)

data.fillna(value=0,inplace=True)

/tmp/ipykernel_81094/2735378082.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.Inflation.fillna(method='ffill',inplace=True)
/tmp/ipykernel_81094/2735378082.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.Inflation.fillna(method='ffill',inplace=True)
/tmp/ipykernel_81094/2735378082.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has

In [53]:
# Define election dates
election_date= datetime.strptime('2024-07-04', '%Y-%m-%d')
cutoff_date = election_date - timedelta(days=52)
prediction_date = election_date - timedelta(days=22)

In [54]:
# Split data
data_train = data[data['startdate'] > '2003-12-31']
data_train = data_train[data_train['startdate'] < cutoff_date]
data_test_1 = data[(data['startdate'] >= cutoff_date) & (data['startdate'] < prediction_date)]
data_test = data_test_1[data_test_1['next_elec_date'] == election_date]

In [55]:
# Fit transform preprocessing pipeline to data_train
data_train_processed = preproc_pipeline.fit_transform(data_train)

In [56]:
# Transform preprocessing pipeline to data_test
data_test_processed = preproc_pipeline.transform(data_test)

In [57]:
# Check feature names make sense for data_train
data_train_processed = pd.DataFrame(
    data_train_processed, columns=preproc_pipeline.get_feature_names_out()
)

# Check feature names make sense for data_test
data_test_processed = pd.DataFrame(
    data_test_processed, columns=preproc_pipeline.get_feature_names_out()
)

In [58]:
data_train_processed.columns

Index(['samplesize', 'months_to_elec_weight', 'GDP', 'Inflation',
       'Unemployment', 'rating', 'party_in_power_Conservative',
       'party_in_power_Conservative_Liberal', 'party_in_power_Labour',
       'Unnamed: 0', 'startdate', 'enddate', 'pollster', 'next_elec_date',
       'days_to_elec', 'months_to_elec', 'poll_length', 'CON_FC', 'LAB_FC',
       'LIB_FC', 'BRX_FC', 'GRE_FC', 'OTH_FC', 'PLC_FC', 'SNP_FC', 'UKI_FC',
       'CON_ACT', 'LAB_ACT', 'LIB_ACT', 'BRX_ACT', 'GRE_ACT', 'PLC_ACT',
       'SNP_ACT', 'UKI_ACT', 'OTH_ACT', 'enddate_year_month', 'Month_x',
       'LAB_trends', 'CON_trends', 'LIB_trends', 'GRE_trends', 'BRX_trends',
       'PLC_trends', 'SNP_trends', 'UKI_trends', 'NAT_trends', 'Month_y'],
      dtype='object')

In [59]:
X_columns = ['samplesize', 'months_to_elec_weight','Inflation','rating','party_in_power_Conservative',
       'party_in_power_Conservative_Liberal', 'poll_length', 'CON_FC',
       'LAB_FC', 'LIB_FC', 'BRX_FC', 'GRE_FC', 'OTH_FC', 'PLC_FC', 'SNP_FC','UKI_FC','LAB_trends', 'CON_trends', 'LIB_trends', 'GRE_trends',
       'BRX_trends', 'PLC_trends', 'SNP_trends', 'UKI_trends']

# Define our X by dropping irrelevant and y columns
X_train = data_train_processed[X_columns]
X_test = data_test_processed[X_columns]

In [60]:
X_train.columns

Index(['samplesize', 'months_to_elec_weight', 'Inflation', 'rating',
       'party_in_power_Conservative', 'party_in_power_Conservative_Liberal',
       'poll_length', 'CON_FC', 'LAB_FC', 'LIB_FC', 'BRX_FC', 'GRE_FC',
       'OTH_FC', 'PLC_FC', 'SNP_FC', 'UKI_FC', 'LAB_trends', 'CON_trends',
       'LIB_trends', 'GRE_trends', 'BRX_trends', 'PLC_trends', 'SNP_trends',
       'UKI_trends'],
      dtype='object')

In [61]:
# Build our target matrix
y_train = data_train_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]
y_test = data_test_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]

In [62]:
# Drops y_train actuals where the actual is the actual result we are trying to predict, replaces with NaNs
y_train.loc[y_train['next_elec_date'] == '2024-07-04',
         ['LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']] = np.nan

In [63]:
# Calculates average median value of X_test
averages = X_test.mean()
averages

samplesize                             0.034061
months_to_elec_weight                  0.773368
Inflation                              0.297872
rating                                 0.638655
party_in_power_Conservative                 1.0
party_in_power_Conservative_Liberal         0.0
poll_length                            2.156863
CON_FC                                  0.23098
LAB_FC                                 0.446471
LIB_FC                                 0.093529
BRX_FC                                     0.12
GRE_FC                                 0.057843
OTH_FC                                 0.018824
PLC_FC                                  0.00451
SNP_FC                                 0.026471
UKI_FC                                      0.0
LAB_trends                                 0.38
CON_trends                             0.108235
LIB_trends                             0.090588
GRE_trends                                 0.03
BRX_trends                             0

In [64]:
# Creates imputation values for y_train to impute over actuals we are trying to predict
imputation_values = {
    'CON_ACT': averages['CON_FC'],
    'LAB_ACT': averages['LAB_FC'],
    'LIB_ACT': averages['LIB_FC'],
    'BRX_ACT': averages['BRX_FC'],
    'GRE_ACT': averages['GRE_FC'],
    'OTH_ACT': averages['OTH_FC'],
    'PLC_ACT': averages['PLC_FC'],
    'SNP_ACT': averages['SNP_FC'],
    'UKI_ACT': averages['UKI_FC']
}

In [65]:
# Applies imputation values to y_train
y_train = y_train.fillna(value=imputation_values)


/tmp/ipykernel_81094/3014033841.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = y_train.fillna(value=imputation_values)


In [66]:
# Instantiate the model for each party
models = {}
parties = ['CON', 'LAB', 'LIB', 'BRX', 'GRE', 'SNP', 'UKI', 'PLC', 'OTH']
for party in parties:
    models[party] = XGBRegressor(
        learning_rate=0.3, n_estimators=300, max_depth=3, subsample=0.7,
        objective='reg:squarederror', nthread=-1, enable_categorical=True
    )

In [67]:
# Train models
X_train_matrix = np.array(X_train)
for party in parties:
    models[party].fit(X_train_matrix, y_train[f'{party}_ACT'])

In [68]:
# actuals_2010 = {
#     "CON": 0.362,  # Conservative Party
#     "LAB": 0.292,  # Labour Party
#     "LIB": 0.235,  # Liberal Democrats
#     "BRX": 0.000,  # Brexit Party (did not exist in 2010)
#     "GRE": 0.009,  # Green Party
#     "SNP": 0.019,  # Scottish National Party
#     "UKI": 0.030,  # UK Independence Party
#     "PLC": 0.005,  # Plaid Cymru
#     "OTH": 0.048   # Other parties
# }

In [69]:
# Evaluate predictions
X_test_matrix = np.array(X_test)
mean_predictions = {}
for party in parties:
    mean_predictions[party] = models[party].predict(X_test_matrix).mean()


In [70]:
# Print mean predictions
for party in parties:
    print(f"2024 mean prediction for {party}: {mean_predictions[party]}")


2024 mean prediction for CON: 0.2812868654727936
2024 mean prediction for LAB: 0.4189617335796356
2024 mean prediction for LIB: 0.0911095067858696
2024 mean prediction for BRX: 0.1079157367348671
2024 mean prediction for GRE: 0.05209532752633095
2024 mean prediction for SNP: 0.029323190450668335
2024 mean prediction for UKI: -0.0009655708563514054
2024 mean prediction for PLC: 0.004630419425666332
2024 mean prediction for OTH: 0.024394238367676735


In [71]:
# Evaluate last five polls
last_predictions = {}
for party in parties:
    last_predictions[party] = models[party].predict(X_test_matrix)[-1]

In [72]:
# Print mean of predictions
for party in parties:
    print(f"2024 last prediction for {party}: {last_predictions[party].mean()}")


2024 last prediction for CON: 0.3415892720222473
2024 last prediction for LAB: 0.3775100111961365
2024 last prediction for LIB: 0.09028789401054382
2024 last prediction for BRX: 0.09602788835763931
2024 last prediction for GRE: 0.04605710133910179
2024 last prediction for SNP: 0.03375572711229324
2024 last prediction for UKI: -0.002774973167106509
2024 last prediction for PLC: 0.004693176597356796
2024 last prediction for OTH: 0.032475072890520096
